In [1]:
import robotic as ry
import numpy as np
import time
print('version:', ry.__version__, ry.compiled())

version: 0.2.2 compile time: Nov 26 2024 16:47:22


In [2]:
C = ry.Config()
C.addFile("maze.g")
C.addFile("cargo2.g")
#C.view()

In [3]:
ego = C.getFrame("cargo2")

end_pos = C.getFrame("goal_area").getPosition()
start_pos = C.getFrame("cargo2").getPosition()

q_start = ego.getJointState()
print("q_start: ",q_start)

ego.setPosition(end_pos)
q_end = ego.getJointState()

print("q_end: ",q_end)


ego.setPosition(start_pos)
C.view()

q_start:  [0.5 1. ]
q_end:  [-4.5 -3. ]


0

In [4]:
#RRT Part

In [5]:
# rrt function, returns the pathways.
def rrt_path(q_start, q_end):
    for i in range(5):
        rrt = ry.PathFinder()
        rrt.setProblem(C, [q_start], [q_end])
        result = rrt.solve()
        print("result.feasible: ",result.feasible)
        if result.feasible:
            return result.x
    raise ValueError("No feasible path found after 5 attempts. Please restart the kernel")

In [6]:
#viewing the solution
path_to_goal = rrt_path(q_start, q_end)

#for config in path_to_goal:
#    C.setJointState(config)
#    C.view()
#    time.sleep(0.1)

result.feasible:  True


In [7]:
#print(path_to_goal) # rrt returns an array of points
del C

In [8]:
# reconfiguration
C = ry.Config()
C.addFile("maze.g")
C.addFile("cargo.g")
C.addFile("ego.g")
C.view()

0

In [9]:
# PART 2    
path_to_goal_3d = []
for i, point in enumerate(path_to_goal):
    # Add a z-coordinate of 0.0 to each point
    path_to_goal_3d.append([point[0], point[1], np.float64(0.0)])
    f = C.addFrame(f"waypoint{i}")
    f.setPosition(path_to_goal_3d[i])
print(path_to_goal_3d)

[[np.float64(0.5), np.float64(1.0), np.float64(0.0)], [np.float64(0.4211513980575041), np.float64(0.9384947321628965), np.float64(0.0)], [np.float64(0.32289394314492387), np.float64(0.9199078401660548), np.float64(0.0)], [np.float64(0.24529283256579698), np.float64(0.8568359158503489), np.float64(0.0)], [np.float64(0.16769963282064138), np.float64(0.7937542596156845), np.float64(0.0)], [np.float64(0.09100785168881123), np.float64(0.7295797264516055), np.float64(0.0)], [np.float64(0.01339901996112769), np.float64(0.6665173031088247), np.float64(0.0)], [np.float64(-0.06421759289649573), np.float64(0.6034644569517336), np.float64(0.0)], [np.float64(-0.1408732683883332), np.float64(0.5392468003852556), np.float64(0.0)], [np.float64(-0.2175302323028735), np.float64(0.47503068182141783), np.float64(0.0)], [np.float64(-0.294154351719945), np.float64(0.41077537602058634), np.float64(0.0)], [np.float64(-0.37002923342180827), np.float64(0.3456370437827384), np.float64(0.0)], [np.float64(-0.44772

In [13]:
# for i in range(len(path_to_goal)): # for each waypoint in rrt
komo = ry.KOMO(C, phases=3, slicesPerPhase=30, kOrder=2, enableCollisions=True)  
    
komo.addControlObjective([], 0)
komo.addControlObjective([], 2)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)

komo.addObjective([1], ry.FS.distance, ["ego1","cargo"],ry.OT.eq)
komo.addModeSwitch([1,-1], ry.SY.stable, ["ego1", "cargo"], True)

# solve
ret = ry.NLP_Solver(komo.nlp(), verbose=0).solve() 

print(ret)
print(ret.feasible)
komo.view_play(True, f"Solution, {ret.feasible}, {ret.eq}")
q = komo.getPath()
print(q.shape)

R = komo.report(False, plotOverTime=True)

{ time: 0.018483, evals: 7, done: 1, feasible: 0, eq: 0.109336, ineq: 0, sos: 17.1969, f: 0 }
False
(90, 4)


In [14]:
del C

In [18]:
# ego-1
C_segment1 = ry.Config()
C_segment1.addFile("maze.g")
C_segment1.addFile("cargo.g")
C_segment1.addFile("ego.g")
#C_segment1.view()

current_holder = None

#for i in range(len(path_to_goal)): # for each waypoint in rrt
komo_ego1 = ry.KOMO(C_segment1, phases=4, slicesPerPhase=30, kOrder=2, enableCollisions=True)  

komo_ego1.addControlObjective([], 0)
komo_ego1.addControlObjective([], 2)
komo_ego1.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)

if current_holder != "ego1": # if the current holder is not ego1
    komo_ego1.addObjective([1], ry.FS.distance, ["ego1","cargo"],ry.OT.eq)
    komo_ego1.addModeSwitch([1,-1], ry.SY.stable, ["ego1", "cargo"], True)
else: # already holding
    komo_ego1.addModeSwitch([1,-1], ry.SY.stable, ["ego1", "cargo"], True)
    
komo_ego1.addObjective([3], ry.FS.position, ["cargo"], ry.OT.eq, [1e2], path_to_goal_3d[2])

# solve
ret1 = ry.NLP_Solver(komo_ego1.nlp(), verbose=2).solve() 

komo_ego1.view_play(True, f"Solution, {ret1.feasible}, {ret1.eq}")

cost1 = ret1.eq

del C_segment1

In [19]:
# ego-2
C_segment2 = ry.Config()
C_segment2.addFile("maze.g")
C_segment2.addFile("cargo.g")
C_segment2.addFile("ego.g")
#C_segment2.view()


#for i in range(len(path_to_goal)): # for each waypoint in rrt
komo_ego2 = ry.KOMO(C_segment2, phases=3, slicesPerPhase=30, kOrder=2, enableCollisions=True)  
    
komo_ego2.addControlObjective([], 0)
komo_ego2.addControlObjective([], 2)
komo_ego2.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)

if current_holder != "ego2": # if the current holder is not ego1
    komo_ego2.addObjective([1], ry.FS.distance, ["ego2","cargo"],ry.OT.eq)
    komo_ego2.addModeSwitch([1,-1], ry.SY.stable, ["ego2", "cargo"], True)
else: # already holding
    komo_ego2.addModeSwitch([1,-1], ry.SY.stable, ["ego2", "cargo"], True)
    
komo_ego2.addObjective([3], ry.FS.position, ["cargo"], ry.OT.eq, [1e2], path_to_goal_3d[2])

# solve
ret2 = ry.NLP_Solver(komo_ego2.nlp(), verbose=2).solve() 

komo_ego2.view_play(True, f"Solution, {ret2.feasible}, {ret2.eq}")

cost2 = ret2.eq

del C_segment2

In [19]:
C = ry.Config()
C.addFile("maze.g")
C.addFile("cargo.g")
C.addFile("ego.g")
C.view()

# 1 komo, we use Stable all the time. and 
total_path = []

C_segment1 = C
segment_path = []



print("cost1:",cost1, "cost2:",cost2, "ret1.feasible:",ret1.feasible,"ret2.feasible:",ret2.feasible)

if cost1 < cost2:
    segment_path = komo_ego1.getPath()
    current_holder = "ego1"
    C = C_segment1
else:
    segment_path = komo_ego2.getPath()
    current_holder = "ego2"
    C = C_segment2

C.view()

C.setJointState(segment_path[-1])
total_path.extend(segment_path)
    
# Visualize the entire movement
for i, config in enumerate(total_path):
    C.setJointState(config)
    C.view(False, f"Step {i}")
    time.sleep(0.1)
R = komo_ego1.report(False, plotOverTime=True)

cost1: 11.21750720548711 cost2: 11.34789839237115 ret1.feasible: False ret2.feasible: False


In [108]:
print(ret)
print(ret.feasible)
komo.view_play(True, f"Solution, {ret.feasible}, {ret.eq}")
q = komo.getPath()
print(q.shape)

{ time: 0.291095, evals: 95, done: 1, feasible: 0, eq: 13.6088, ineq: 0, sos: 16.2845, f: 0 }
False
STACK9 /usr/bin/python3(+0x198a6b) [0x5bd41c3a5a6b]
STACK8 /usr/bin/python3(_PyObject_MakeTpCall
STACK7 /usr/bin/python3(+0x18ae12) [0x5bd41c397e12]
STACK6 /home/omer/.local/lib/python3.10/site-packages/robotic/_robotic.so(+0x3f21e) [0x7b3ca0f2721e]
STACK5 /home/omer/.local/lib/python3.10/site-packages/robotic/_robotic.so(+0x8f6ac) [0x7b3ca0f776ac]
STACK4 KOMO::view_play(bool, char const*, double, char const*)
STACK3 KOMO::view(bool, char const*)
STACK2 KOMO::info_objectiveErrorTraces()
STACK1 rai::Array<double>::operator()(int) const
STACK0 rai::LogToken::~LogToken()


== ERROR:array.ipp:operator():738(-2) CHECK failed: 'nd==1 && (uint)i<d0' -- 1D range error (1=1, 480<480)


RuntimeError: array.ipp:operator():738(-2) CHECK failed: 'nd==1 && (uint)i<d0' -- 1D range error (1=1, 480<480)

In [82]:
R = komo.report(False, plotOverTime=True)